<a href="https://colab.research.google.com/github/zayednetad/gpu_programming/blob/main/HomePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**An example of how to use the model?**
1. Download the train dataset from https://www.kaggle.com/datasets/nickptaylor/iowa-house-prices
2. Upload to colab, the typical path is /content/train.csv
3. Then perform ML training on the IOWA House Pricing Data and save the model
4. We will use Flask to create website to use the model for prediction of new house data, first we need to install related python packages
5. We will use ngrok auth token to able to launch the created Flask website
6. Then we will run the python flask code to create and launch the website
7. We will provide input data to predict home price

In [16]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import joblib

# Load the data
mlb_file_path = '/content/train.csv'
home_data = pd.read_csv(mlb_file_path)

# Define features and target
y = home_data.SalePrice
feature_names = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[feature_names]

# Train the model
iowa_model = DecisionTreeRegressor(random_state=1)
iowa_model.fit(X, y)

# Save the model to file
joblib.dump(iowa_model, 'iowa_model.pkl')
print("Model saved as 'iowa_model.pkl'")

Model saved as 'iowa_model.pkl'


In [17]:
!pip install flask pyngrok joblib scikit-learn

In [18]:
from pyngrok import conf
conf.get_default().auth_token = "1abGKpas61AzmZsytmGUWgb8f1r_3eRYsdkrZtpi6wV7KtRGc"

In [19]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import joblib
import numpy as np

# Load your model
model = joblib.load("/content/iowa_model.pkl")

# Start Flask app
app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head><title>House Price Predictor</title></head>
<body>
    <h2>Enter House Features</h2>
    <form method="POST">
        <label>Lot Area: </label><input type="text" name="LotArea"><br><br>
        <label>Year Built: </label><input type="text" name="YearBuilt"><br><br>
        <label>1st Floor SF: </label><input type="text" name="1stFlrSF"><br><br>
        <label>2nd Floor SF: </label><input type="text" name="2ndFlrSF"><br><br>
        <label>Full Bath: </label><input type="text" name="FullBath"><br><br>
        <label>Bedrooms Above Ground: </label><input type="text" name="BedroomAbvGr"><br><br>
        <label>Total Rooms Above Ground: </label><input type="text" name="TotRmsAbvGrd"><br><br>
        <input type="submit" value="Predict Sale Price">
    </form>
    {% if prediction %}
        <h3>Predicted Sale Price: {{ prediction }}</h3>
    {% endif %}
</body>
</html>
'''

@app.route("/", methods=["GET", "POST"])
def predict():
    prediction = None
    if request.method == 'POST':
        try:
            input_data = np.array([[
                float(request.form['LotArea']),
                int(request.form['YearBuilt']),
                float(request.form['1stFlrSF']),
                float(request.form['2ndFlrSF']),
                int(request.form['FullBath']),
                int(request.form['BedroomAbvGr']),
                int(request.form['TotRmsAbvGrd'])
            ]])
            prediction = round(model.predict(input_data)[0], 2)
        except Exception as e:
            prediction = f"Error: {str(e)}"

    return render_template_string(HTML_TEMPLATE, prediction=prediction)

# 🔥 Expose via public URL
public_url = ngrok.connect(5000)
print("🚀 Flask app running at:", public_url)

app.run(port=5000)

🚀 Flask app running at: NgrokTunnel: "https://fb78-35-198-201-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/May/2025 21:33:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 21:33:46] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [16/May/2025 21:36:34] "POST / HTTP/1.1" 200 -
